# Sequence Alignment
Optimization problem, pairwise sequence alignment, two sequences (finite alphabet), objective function.

The end goal is an optimal alignment (maximizes the objective function)

The objective function is additive for (DNA & RNA):
 * `matches: +2/+3` (positive contribution)
 * `mismatches: -2/-3` (replacement, negative contribution)
 * `gaps: -2/-3`(higher impact than replacement, negative contribution)

Matrices used for these topics:
 * **Substitution matrix** -> assigns a value for each pair, does not consider gaps
 * **PAM matrix** -> provides the score of the substitution of one aminoacid by another
 * **BLOSUM** (BLOcks SUbstitution Matrix) -> also a substitution matrix for sequence alignment but is based on frequencies of the proteins
 
The objective function is additive for (aminoacids):
 * `gaps: -12 to -7 = g + r * len` where `g` is the cost of gap opening, `r` is the cost of each gap and `len` is the extension of the gap (how many symbols), negative value 

In [68]:
# returns a dict of AA -> match_value (or mismatch) for each pair of symbols
def create_submat(match, mismatch, alphabet="ACGT"):
    return {x+y: match if x==y else mismatch for x in alphabet for y in alphabet}

In [69]:
print(create_submat(1, 0))

{'AA': 1, 'AC': 0, 'AG': 0, 'AT': 0, 'CA': 0, 'CC': 1, 'CG': 0, 'CT': 0, 'GA': 0, 'GC': 0, 'GG': 1, 'GT': 0, 'TA': 0, 'TC': 0, 'TG': 0, 'TT': 1}


### Read Substitution Matrix

In [22]:
import pandas as pd

In [83]:
def read_submat(filename="blosum62.mat"):
    df = pd.read_csv(filename, header=0, delimiter="\t")
    alphabet = list(df.columns)
    sm = {x+y:df.loc[i][j]  for i,x in enumerate(alphabet) for j,y in enumerate(alphabet)}
    return df, alphabet, sm

In [84]:
df, alf, sm = read_submat()
df

,A,R,N,D,C,Q,E,G,H,I,L,K,M,F,P,S,T,W,Y,V
0,4,-1,-2,-2,0,-1,-1,0,-2,-1,-1,-1,-1,-2,-1,1,0,-3,-2,0
1,-1,5,0,-2,-3,1,0,-2,0,-3,-2,2,-1,-3,-2,-1,-1,-3,-2,-3
2,-2,0,6,1,-3,0,0,0,1,-3,-3,0,-2,-3,-2,1,0,-4,-2,-3
3,-2,-2,1,6,-3,0,2,-1,-1,-3,-4,-1,-3,-3,-1,0,-1,-4,-3,-3
4,0,-3,-3,-3,9,-3,-4,-3,-3,-1,-1,-3,-1,-2,-3,-1,-1,-2,-2,-1
5,-1,1,0,0,-3,5,2,-2,0,-3,-2,1,0,-3,-1,0,-1,-2,-1,-2
6,-1,0,0,2,-4,2,5,-2,0,-3,-3,1,-2,-3,-1,0,-1,-3,-2,-2
7,0,-2,0,-1,-3,-2,-2,6,-2,-4,-4,-2,-3,-3,-2,0,-2,-2,-3,-3
8,-2,0,1,-1,-3,0,0,-2,8,-3,-3,-1,-2,-1,-2,-1,-2,-2,2,-3
9,-1,-3,-3,-3,-1,-3,-3,-4,-3,4,2,-3,1,0,-3,-2,-1,-3,-1,3


In [76]:
def score_column(c1, c2, sm, g): # g is the gap penalty
    if c1 == "_" or c2 == "_": return g
    return sm[c1+c2]

In [77]:
assert score_column("A", "R", sm, -10) == -1, "Score AR should be -1"
assert score_column("A", "_", sm, -10) == -10, "Score A_ should be gap penalty -10"
assert score_column("_", "A", sm, -10) == -10, "Score _A should be gap penalty -10"

In [78]:
def score_seq(s1, s2, sm, g): 
    assert len(s1) == len(s2), "Sequences 1 and 2 should have the same length, at least for now"
    return sum(score_column(s1[i],s2[i], sm, g) for i in range(len(s1)))    

In [79]:
assert score_seq("ARN_AKF", "AR_ALKF", sm, -10) == -1, "should be -1"
assert score_seq("AR_ALF", "AR_ALF", sm, -15) == 8, "should be 8"

## Testing with teacher's input

In [81]:
score_seq("_CAGTGCATG_ACATA", "TCAG_GC_TCTACAGA", create_submat(2, -2), -3)

4

In [85]:
score_seq("LGPSSGCASRIWTKSA", "TGPS_G__S_IWSKSG", read_submat()[2], -8)

19

### Affine gap score
This involves counting more for larger gaps...